In [1]:
import pandas as pd
import numpy as np
from modules import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt
from nltk.util import everygrams

In [16]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘']) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [17]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
name_list =["술 리뷰(곰표막걸리)","술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]

In [18]:
all_data = pd.DataFrame()

for name in name_list:
    data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    data.columns = ['date', 'star','doc']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


C:\Users\user\AppData\Local\Temp\ipykernel_16296\639027951.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16296\639027951.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16296\639027951.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16296\639027951.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_16296\639027951.py:8: FutureWarning: The defa

In [19]:
all_data

,date,star,doc
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요
...,...,...,...
5674,2020-04-03,5,다양한술을 집에서 편하게 주문가능하기에 자주 주문합니다이번에도 즐겁게 마시겠읍니다
5675,2020-04-03,5,빠른 배송 감사합니당
5676,2020-03-31,5,단맛은 있는데 꿀 단맛이어서 나쁘지 않아요 기분 나쁘지 않은 단맛이에요 인공감미료가...
5677,2020-03-26,5,맛이 달고 청량감이 좋아요


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
# 특징을 추출할 문서의 tf-idf를 구함
a = "술 리뷰(복순도가)"
bow_rep_tfidf = tfidf_fit.transform(globals()[a].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출

word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>4.9][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>420]

C:\Users\user\AppData\Local\Temp\ipykernel_16296\3357611957.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>4.9][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>420]


,단어,tf-idf,idf
5489,샴페인,223.865030,5.501938
4793,부모님,213.755274,5.122699
3375,많다,206.915492,4.947088
268,강하다,197.670808,5.159147
114,가족,173.578981,5.283560
9885,추천,167.900577,4.978514
9309,지인,166.742207,5.068860
